In [40]:
import copy
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display
from IPython.display import display, clear_output

class Individuo:
    def __init__(self, solucion, velocidad):
        self._solucion = solucion
        self._velocidad = velocidad
        self._b = copy.deepcopy(solucion)
        self._b_fitness = np.inf

In [41]:
class PSO:
    def __init__(self,
    cantidad_individuos,
    dimensiones,
    ro, #Tamaño de vecindad
    phi1_max,
    phi2_max,
    v_max,
    problema,
    generaciones):
        self._cantidad_individuos = cantidad_individuos
        self._dimensiones = dimensiones
        self._ro = ro
        self._phi1_max = phi1_max
        self._phi2_max = phi2_max
        self._v_max = v_max
        self._problema = problema
        self._generaciones = generaciones
        self._individuos = []
        self._rango = self._problema.MAX_VALUE - self._problema.MIN_VALUE
        self._mejor = np.inf

    def crearIndividuos(self):
        for i in range(self._cantidad_individuos):
            solucion = np.random.random(size = self._dimensiones) * self._rango + self._problema.MIN_VALUE
            velocidad = np.random.random(size = self._dimensiones) * self._v_max * 2 + self._v_max
            individuo = Individuo(solucion, velocidad)
            individuo._b_fitness = self._problema.fitness(individuo._solucion)
            self._individuos.append(individuo)

    def mejorIndividuo(self):
        for i in self._individuos:
            fitness = self._problema.fitness(i._solucion)
            if fitness < self._mejor:
                self._mejor = fitness

    def run(self):
        self.crearIndividuos()
        self.mejorIndividuo()
        mejores_historicos = [] #Para los promedios
        generacion = 0
        while (generacion <= self._generaciones):
            for idx in range(len(self._individuos)):
                h = 0
                for i in range(-self._ro // 2, self._ro // 2 + 1):
                    if i == 0:
                        continue
                    elif i == -self._ro // 2:
                        h = copy.deepcopy(self._individuos[(idx + i) % len(self._individuos)])
                    elif self._problema.fitness(self._individuos[(idx + i) % len(self._individuos)]._solucion) < self._problema.fitness(h._solucion):
                        h = copy.deepcopy(self._individuos[(idx + i) % len(self._individuos)])
                phi1 = np.random.random(size = self._dimensiones) * self._phi1_max
                phi2 = np.random.random(size = self._dimensiones) * self._phi2_max
                self._individuos[idx]._velocidad = (self._individuos[idx]._velocidad +
                np.multiply(phi1, self._individuos[idx]._b - self._individuos[idx]._solucion) +
                np.multiply(phi2, h._solucion - self._individuos[idx]._solucion))
                for i in range(self._dimensiones):
                    if abs(self._individuos[idx]._velocidad[i]) > self._v_max:
                        self._individuos[idx]._velocidad[i] = self._v_max / (self._individuos[idx]._velocidad[i])
                self._individuos[idx]._solucion = self._individuos[idx]._solucion + self._individuos[idx]._velocidad
                fitness_individuo = self._problema.fitness(self._individuos[idx]._solucion)
                if fitness_individuo < self._individuos[idx]._b_fitness:
                    self._individuos[idx]._b = copy.deepcopy(self._individuos[idx]._solucion)
                    self._individuos[idx]._b_fitness = fitness_individuo
                    if fitness_individuo < self._mejor:
                        self._mejor = fitness_individuo

            if generacion % 100 == 0:
                #Guardar los mejores historicos para promediar y graficar
                mejores_historicos.append(abs(self._mejor))
                print('Generación ', generacion, ':', self._mejor)
            generacion += 1
        return mejores_historicos

In [42]:
#Funciones
class Sphere:
  MIN_VALUE = -5.12
  MAX_VALUE = 5.12
  def __init__(self):
      pass
  def fitness(self, cromosoma):
      z = 0
      for alelo in cromosoma:
          z += alelo**2
      return z

class Rosenbrock:
  MIN_VALUE = -2.048
  MAX_VALUE = 2.048
  def __init__(self):
    pass
  def fitness(self, vector):
    z = 0
    for dimension in range(len(vector)-1):
      z += 100 * (vector[dimension + 1] - vector[dimension]**2)**2 + (vector[dimension]-1)**2
    return z

class Rastrigin:
  MIN_VALUE = -5.12
  MAX_VALUE = 5.12
  def __init__(self):
      pass
  def fitness(self, cromosoma, C = 10):
    z = C * len(cromosoma) #10n
    for alelo in cromosoma:
      z += alelo**2 - C * np.cos(2 * np.pi * alelo)
    return z
class Quartic:
  MIN_VALUE = -1.28
  MAX_VALUE = 1.28
  def __init__(self):
      pass
  def fitness(self, cromosoma):
    z = 0
    for alelo in cromosoma:
      z += alelo**4
    return z

In [46]:
#Función para graficar los promedios individualmente
def graficarIndividual(arreglo, nombre, dimensiones):
  datos_y = arreglo
  datos_x = range(len(datos_y))

  # Graficar la línea
  plt.plot(datos_x, datos_y, marker='o')

  plt.xticks(range(0, 20))

  plt.title(f'{nombre}: {dimensiones} dimensiones')

  # Agregar etiquetas a los ejes
  plt.xlabel('Eje X')
  plt.ylabel('Eje Y')

  # Mostrar la gráfica
  plt.show()

In [57]:
#Función para graficae los promedios en conjunto
def graficarConjunto(arregloPromedios):
    colores = ['red', 'blue', 'green']
    dimensiones = [2, 4, 8]
    datos_x = range(len(arregloPromedios[0]))  # asumimos que todos los arreglos tienen la misma longitud

    # Verificamos si algún valor en los arreglos internos es mayor que 100
    valorMaximo = any(max(arreglo) > 100 for arreglo in arregloPromedios)

    if valorMaximo:
      plt.ylim(0, 100)  # Establece el límite superior en el eje Y a 100
    #---------------------------------------------------------------------

    # Graficar cada arreglo en la misma gráfica con colores diferentes
    for i, arreglo in enumerate(arregloPromedios):
        color = colores[i] if colores else None
        plt.plot(datos_x, arreglo, marker='o', label=f'{dimensiones[i]} dimensiones', color=color)

    plt.xticks(range(0, 20))

    plt.title('Promedios de todas las dimensiones')

    # Agregar etiquetas a los ejes
    plt.xlabel('Eje X')
    plt.ylabel('Eje Y')

    # Mostrar la gráfica
    plt.legend()
    plt.show()

In [58]:
def main():
  # Funciones de optimización disponibles
  optimization_functions = {
    'Sphere': Sphere(),
    'Rosenbrock': Rosenbrock(),
    'Rastrigin': Rastrigin(),
    'Quartic': Quartic()
  }

  # Arreglo para graficar los promedios en conjunto
  conjuntoPromedios = []
  # Dimensiones
  availableDimensions = [2, 4, 8]

  # Widget de selección de función
  function_selector = widgets.Dropdown(
    options=list(optimization_functions.keys()),
    value='Sphere',
    description='Función:'
  )

  # Función que se llama al hacer clic en el botón
  def on_button_click(b):
    output.clear_output()
    selected_function_name = function_selector.value
    selected_function_class = optimization_functions[selected_function_name]
    selected_function_instance = optimization_functions[selected_function_name]
    rango = selected_function_instance.MAX_VALUE - selected_function_instance.MIN_VALUE
    cantidad_individuos = 30
    ro = 8
    phi1_max = 1.7
    phi2_max = 2.0
    v_max = rango * 0.01
    generaciones = 2000

    with output:
        # Ejecutar la función para cada dimensión disponible
        for dimensions in availableDimensions:
          mejores_historicosG = []
          print(f"\nFunción seleccionada: {function_selector.value}")
          print(f"Dimensiones: {dimensions}")
          for i in range(5):
              print(f"\nRun número {i+1}")
              pso = PSO(cantidad_individuos, dimensions, ro, phi1_max, phi2_max, v_max, selected_function_instance, generaciones)
              mejores_historicosG.append(pso.run())
          #Promediar
          mejores_historicosG = [sum(x) / len(x) for x in zip(*mejores_historicosG)]
          conjuntoPromedios.append(mejores_historicosG)
          print(f'\nPromedios de los mejores historicos: \n{mejores_historicosG}\n')
          #Graficar individualmente
          graficarIndividual(mejores_historicosG, function_selector.value, dimensions)
        # Graficar promedios en conjunto
        print('\nGráica con todos los promedios:\n')
        graficarConjunto(conjuntoPromedios)

  # Widget de botón
  button = widgets.Button(description="Ejecutar")
  output = widgets.Output()

  # Asignar la función al evento de clic del botón
  button.on_click(on_button_click)

  # Mostrar widgets
  display(function_selector, button, output)